*** Identify a domain of interest : health, business, retail, education etc.
    
    --- My domain of interest is HEALTH.
    

*** Think out ways AI(data science) can be used to solve a problem in your chosen domain of interest.

    ===  In the domain of Health, AI and data science can be immensely beneficial in numerous ways. Here are several ways AI can be used to solve problems in healthcare:
   
   1. Disease Diagnosis and Prediction
   
   2. Personalized Treatment Plans
   
   3. Drug Discovery and Development
   
   4. Remote Patient Monitoring
   
   5. Healthcare Resource Optimization
   
   6. Medical Imaging Analysis
   
   7. Patient Engagement and Education
   
   8. Healthcare Fraud Detection
   
     The integration of AI technologies holds the potential to revolutionize various aspects of healthcare delivery, ultimately leading to better patient care and outcomes.  
   

## Predicting Food Price Trends in Nigeria Model

* Problem Statement
 
   The goal is to provide insights into future food prices, which can aid stakeholders such as policymakers, farmers, and consumers in making informed decisions regarding food production, distribution, and consumption.
 
 
* Approach

  * Data Preprocessing
  * Feature Engineering
  * Model Selection
  * Model Training
  * Model Evaluation
  * Prediction
  
* Expected Outcome
  
  The developed machine learning model will provide accurate predictions of food price trends in Nigeria, enabling stakeholders to make informed decisions and take appropriate actions to address food security challenges, optimize resource allocation, and mitigate the impact of price fluctuations on consumers and producers.    

*** Get this data and identify what you want to model and whether it would be supervised learning or unsupervised learning .

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

In [2]:
df = pd.read_csv('wfp_food_prices_nga.csv')

C:\Users\HP\AppData\Local\Temp\ipykernel_772\3464975351.py:1: DtypeWarning: Columns (4,5,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('wfp_food_prices_nga.csv')


# Data Exploration

In [3]:
df.head()

,date,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice
0,#date,#adm1+name,#adm2+name,#loc+market+name,#geo+lat,#geo+lon,#item+type,#item+name,#item+unit,#item+price+flag,#item+price+type,#currency,#value,#value+usd
1,1/15/2002,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Maize,KG,actual,Wholesale,NGN,175.92,1.5525
2,1/15/2002,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Millet,KG,actual,Wholesale,NGN,150.18,1.3254
3,1/15/2002,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Rice (imported),KG,actual,Wholesale,NGN,358.7,3.1656
4,1/15/2002,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Sorghum,KG,actual,Wholesale,NGN,155.61,1.3733


In [4]:
df.shape

(80982, 14)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80982 entries, 0 to 80981
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       80982 non-null  object
 1   admin1     80982 non-null  object
 2   admin2     80982 non-null  object
 3   market     80982 non-null  object
 4   latitude   80982 non-null  object
 5   longitude  80982 non-null  object
 6   category   80982 non-null  object
 7   commodity  80982 non-null  object
 8   unit       80982 non-null  object
 9   priceflag  80982 non-null  object
 10  pricetype  80982 non-null  object
 11  currency   80982 non-null  object
 12  price      80982 non-null  object
 13  usdprice   80982 non-null  object
dtypes: object(14)
memory usage: 8.6+ MB


In [6]:
df.describe()

,date,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice
count,80982,80982,80982,80982,80982,80982,80982,80982,80982,80982,80982,80982,80982.0,80982.0
unique,265,15,32,41,80,80,9,43,23,5,3,2,20517.0,30489.0
top,11/15/2021,Yobe,Maiduguri,Potiskum,11.7072401,11.08248997,cereals and tubers,Millet,100 KG,actual,Retail,NGN,0.0,0.0
freq,1795,26318,12690,4308,3335,3335,39537,4972,19125,51969,56111,80981,13949.0,13949.0


In [7]:
# Drop unnecessary columns

df = df.drop(columns=['latitude', 'longitude', 'unit', 'priceflag', 'pricetype', 'currency', 'usdprice'])

In [8]:
df.head()

,date,admin1,admin2,market,category,commodity,price
0,#date,#adm1+name,#adm2+name,#loc+market+name,#item+type,#item+name,#value
1,1/15/2002,Katsina,Jibia,Jibia (CBM),cereals and tubers,Maize,175.92
2,1/15/2002,Katsina,Jibia,Jibia (CBM),cereals and tubers,Millet,150.18
3,1/15/2002,Katsina,Jibia,Jibia (CBM),cereals and tubers,Rice (imported),358.7
4,1/15/2002,Katsina,Jibia,Jibia (CBM),cereals and tubers,Sorghum,155.61


* The provided data is already in tabular format, which is suitable for Supervised Learning tasks where we have input features and corresponding target variables. In this case, we can use the data to predict food prices, which would be a Regression Problem.

In [9]:
cat = df.select_dtypes(include=['category', 'object'])
cat_columns = [i for i in cat.columns]

In [10]:
cat_columns

['date', 'admin1', 'admin2', 'market', 'category', 'commodity', 'price']

In [11]:
for i in cat_columns:
    print('-------------------------------')
    print('-------------------------------')
    print(df[i].value_counts())

-------------------------------
-------------------------------
date
11/15/2021    1795
10/15/2021    1780
1/15/2022     1746
3/15/2022     1745
2/15/2022     1745
              ... 
8/15/2004        3
7/15/2004        3
6/15/2004        3
5/15/2004        3
#date            1
Name: count, Length: 265, dtype: int64
-------------------------------
-------------------------------
admin1
Yobe          26318
Borno         21763
Kaduna         4971
Katsina        3773
Kano           3009
Oyo            2846
Zamfara        2759
Jigawa         2746
Kebbi          2723
Gombe          2705
Lagos          2591
Adamawa        2450
Abia           1679
Sokoto          648
#adm1+name        1
Name: count, dtype: int64
-------------------------------
-------------------------------
admin2
Maiduguri       12690
Konduga          6138
Damaturu         4390
Potiskum         4308
Dawakin Tofa     3009
Ibadan North     2846
Kaura Namoda     2759
Gwandu           2723
Akko             2705
Dandume          

In [45]:
df.duplicated().sum()

24648

In [13]:
df.drop_duplicates(inplace=True)

In [46]:
df.duplicated().sum()

24648

In [47]:
df.head()

,date,price,commodity_#item+name,commodity_Bananas,commodity_Beans (niebe),commodity_Beans (red),commodity_Beans (white),commodity_Bread,"commodity_Cassava meal (gari, yellow)",commodity_Cowpeas,...,commodity_Sorghum,commodity_Sorghum (brown),commodity_Sorghum (white),commodity_Spinach,commodity_Sugar,commodity_Tomatoes,commodity_Watermelons,commodity_Wheat,commodity_Yam,commodity_Yam (Abuja)
1,2002-01-15,175.92,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2002-01-15,150.18,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2002-01-15,358.70,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2002-01-15,155.61,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
5,2002-01-15,196.87,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [16]:
df.isnull().sum()

date         0
admin1       0
admin2       0
market       0
category     0
commodity    0
price        0
dtype: int64

In [50]:
df.to_csv('cleaned_wfp_food_prices_nga.csv')

In [51]:
cat_columns

['date', 'admin1', 'admin2', 'market', 'category', 'commodity', 'price']

In [49]:
# Drop any unnecessary columns

df = df.drop(columns=["admin1", "admin2", "market", "category"])

KeyError: "['admin1', 'admin2', 'market', 'category'] not found in axis"

In [52]:
df.head()

,date,price,commodity_#item+name,commodity_Bananas,commodity_Beans (niebe),commodity_Beans (red),commodity_Beans (white),commodity_Bread,"commodity_Cassava meal (gari, yellow)",commodity_Cowpeas,...,commodity_Sorghum,commodity_Sorghum (brown),commodity_Sorghum (white),commodity_Spinach,commodity_Sugar,commodity_Tomatoes,commodity_Watermelons,commodity_Wheat,commodity_Yam,commodity_Yam (Abuja)
1,2002-01-15,175.92,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2002-01-15,150.18,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2002-01-15,358.70,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2002-01-15,155.61,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
5,2002-01-15,196.87,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [53]:
# One-hot encode the commodity column

df = pd.get_dummies(df, columns=['commodity'])

KeyError: "None of [Index(['commodity'], dtype='object')] are in the [columns]"

In [54]:
df.head()

,date,price,commodity_#item+name,commodity_Bananas,commodity_Beans (niebe),commodity_Beans (red),commodity_Beans (white),commodity_Bread,"commodity_Cassava meal (gari, yellow)",commodity_Cowpeas,...,commodity_Sorghum,commodity_Sorghum (brown),commodity_Sorghum (white),commodity_Spinach,commodity_Sugar,commodity_Tomatoes,commodity_Watermelons,commodity_Wheat,commodity_Yam,commodity_Yam (Abuja)
1,2002-01-15,175.92,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2002-01-15,150.18,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2002-01-15,358.70,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2002-01-15,155.61,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
5,2002-01-15,196.87,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77914 entries, 1 to 80981
Data columns (total 45 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   date                                   77914 non-null  datetime64[ns]
 1   price                                  77914 non-null  float64       
 2   commodity_#item+name                   77914 non-null  bool          
 3   commodity_Bananas                      77914 non-null  bool          
 4   commodity_Beans (niebe)                77914 non-null  bool          
 5   commodity_Beans (red)                  77914 non-null  bool          
 6   commodity_Beans (white)                77914 non-null  bool          
 7   commodity_Bread                        77914 non-null  bool          
 8   commodity_Cassava meal (gari, yellow)  77914 non-null  bool          
 9   commodity_Cowpeas                      77914 non-null  bool       

In [24]:
# Convert the date column to datetime

df['date'] = pd.to_datetime(df['date'], errors='coerce')  # Set errors='coerce' to handle any invalid dates

C:\Users\HP\AppData\Local\Temp\ipykernel_772\3244165804.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'], errors='coerce')  # Set errors='coerce' to handle any invalid dates


In [56]:
# Remove rows with missing or invalid dates

df = df.dropna(subset=['date'])

In [57]:
# Convert numerical columns to numeric types, handling errors

for col in df.columns:
    if col != 'date' and col != 'price':
        df[col] = pd.to_numeric(df[col], errors='coerce')

In [58]:
# Remove rows with missing or invalid dates and numeric values

df = df.dropna()

In [59]:
# Convert 'price' column to numeric type, coercing errors

df['price'] = pd.to_numeric(df['price'], errors='coerce')

In [60]:
# Split the data into features (X) and target variable (y)

X = df.drop(columns=['date', 'price'])
y = df['price']

In [61]:
# Split the data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [62]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [32]:
# Model Evaluation


predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)

Mean Squared Error: 86053060.13315032
Mean Absolute Error: 5049.8993117200225


* The MSE represents the average squared difference between the predicted prices and the actual prices. A lower MSE indicates better performance.

*  The MAE represents the average absolute difference between the predicted prices and the actual prices. Similarly, a lower MAE indicates better performance.

In [33]:
from sklearn.linear_model import LinearRegression

# Initialize the Linear Regression model

model = LinearRegression()

# Fit the model to the training data

model.fit(X_train, y_train)

LinearRegression()

In [34]:
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)

Mean Squared Error: 86049972.62363596
Mean Absolute Error: 5051.461366432973


In [35]:
from sklearn.ensemble import GradientBoostingRegressor

# Initialize the Gradient Boosting Regressor model

model = GradientBoostingRegressor(n_estimators=100, random_state=42)

# Fit the model to the training data

model.fit(X_train, y_train)

GradientBoostingRegressor(random_state=42)

In [36]:
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)

Mean Squared Error: 87833511.72518577
Mean Absolute Error: 5468.98937002545


In [37]:
from sklearn.tree import DecisionTreeRegressor

# Initialize the Decision Tree Regressor model

model = DecisionTreeRegressor(random_state=42)

# Fit the model to the training data

model.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [38]:
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)

Mean Squared Error: 86050108.71921735
Mean Absolute Error: 5051.318847533418


In [39]:
# Define the dictionary to store models

models = {
    "Linear Regression": LinearRegression(),
    "Gradient Boosting Regression": GradientBoostingRegressor(n_estimators=100, random_state=42),
    "Random Forest Regression": RandomForestRegressor(n_estimators=100, random_state=42),
    "Decision Tree Regression": DecisionTreeRegressor(random_state=42)
}

In [40]:
# Train and evaluate models

performance = {}

for model_name, model in models.items():
    print("Training", model_name)
    model.fit(X_train, y_train)

Training Linear Regression
Training Gradient Boosting Regression
Training Random Forest Regression
Training Decision Tree Regression


In [41]:
# Model Evaluation

predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
print(model_name + " - Mean Squared Error:", mse)
print(model_name + " - Mean Absolute Error:", mae)
print("-------------------------------------")

Decision Tree Regression - Mean Squared Error: 86050108.71921735
Decision Tree Regression - Mean Absolute Error: 5051.318847533418
-------------------------------------


In [42]:
# Store performance metrics in the dictionary

performance[model_name] = {"MSE": mse, "MAE": mae}

In [43]:
# Identify the best performing model based on MSE or MAE

best_model = min(performance, key=lambda x: performance[x]["MSE"])  # or "MAE"
print("Best Performing Model:", best_model)

Best Performing Model: Decision Tree Regression


In [44]:
import joblib

# Assuming best_model contains the name of the best performing model
best_model = models[best_model]

# Save the best performing model to a file
joblib.dump(best_model, 'best_model.joblib')

['best_model.joblib']